<a href="https://colab.research.google.com/github/carloshvmoraes/ISGDTPDN/blob/main/ISGD_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating Smart Grid Devices into the Traditional Protection of Distribution Networks

## Install Python Libraries

In [1]:
!pip install pandapower tqdm

     |████████████████████████████████| 5.9 MB 4.8 MB/s 
  Created wheel for pandapower: filename=pandapower-2.8.0-py3-none-any.whl size=5881019 sha256=050805f9dc05a1f4dac58037cc72c86099a6eef5bf7cc082f70386257f8d8eeb
  Stored in directory: /root/.cache/pip/wheels/8a/ce/91/cb5881926915c6c131434bde7bb43a9f9e1c7a54951ebd3dee
Successfully built pandapower


## Download Source ISGDTPDN

In [2]:
!wget https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/src/ISGDTPDN.py -O ISGDTPDN.py
!wget https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/examples/CWB_p64.py -O CWB_p64.py

--2022-03-03 21:32:44--  https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/src/ISGDTPDN.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2509 (2.5K) [text/plain]
Saving to: ‘ISGDTPDN.py’

ISGDTPDN.py         100%[===================>]   2.45K  --.-KB/s    in 0s      

2022-03-03 21:32:44 (29.0 MB/s) - ‘ISGDTPDN.py’ saved [2509/2509]

--2022-03-03 21:32:44--  https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/examples/CWB_p64.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5433 (5.3K) [text/plain]
Savi

## Example - CWB Bus 64

<img src="https://github.com/carloshvmoraes/ISGDTPDN/raw/main/examples/CWB_p64.png" width="500" />

## Benchmark

In [3]:
from ISGDTPDN import CurrSaved, CurrSavedNew, FaultDetect
from CWB_p64 import new_network
import pandapower as pp
import random
import pandas as pd
import IPython
import tqdm.notebook as tq


random.seed(42)

# basic settings for 2 scenarios
scenarios = [
            {'sw_meters':[64], 'iteration':1000},
            {'sw_meters':[64, 68], 'iteration':1000},
            ]

for setting in scenarios:
  name_scenario = '-'.join(map(str,setting['sw_meters']))
  display(IPython.display.HTML(f'<h1>Scenario {name_scenario}</h1>'))

  filename = name_scenario + '_' + str(setting['iteration'])
  buffer = open(filename + '_bm.csv','w')
  buffer.write('iter,lr,h1,h2,h3,h4,er\n')

  for LoadRange in [0.01, 0.02, 0.05, 0.1]:
    display(IPython.display.HTML(f'<h2>Load Range: {LoadRange}</h2>'))

    for n in tq.tqdm(range(setting['iteration'])):
      # create distribuition net
      net = new_network(medidores=setting['sw_meters'])

      # Fault Pattern Currents
      I_saved = CurrSaved(net)

      # load change by Range
      net.load['p_mw'] = [random.uniform(1-LoadRange, 1+LoadRange) * r['p_mw'] for _, r in net.load.iterrows()]

      # Fault Pattern correction
      I_saved_new = CurrSavedNew(net, I_saved)

      # solution totalization
      res = [0,0,0,0,0]
      for sw_id, r in net.switch.iterrows():
        # create fault
        net.switch.at[sw_id, 'closed'] = False
        pp.runpp(net, neglect_open_switch_branches=True)
        # remove fault
        net.switch.at[sw_id, 'closed'] = True
        
        # search fault
        fault_best = FaultDetect(net, I_saved_new)
        # best solution
        if sw_id in fault_best:
          index = fault_best.index(sw_id)
          index = index if index < 4 else 4
          res[index] += 1

      res = [n, LoadRange] + res
      buffer.write(','.join(map(str, res)) + '\n')

  buffer.close()

  # buffer totalization - memory saves
  df = pd.read_csv(filename + '_bm.csv')
  df = df.drop(['iter'], axis=1)

  grupos = df.groupby(by='lr').sum()
  grupos = grupos.div(grupos.sum(axis=1), axis=0)
  grupos.to_csv(filename + '_percents.csv')

  display(grupos)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

,h1,h2,h3,h4,er
lr,,,,,
0.01,0.952381,0.042857,0.004762,0.000000,0.000000
0.02,0.961905,0.038095,0.000000,0.000000,0.000000
0.05,0.900000,0.100000,0.000000,0.000000,0.000000
0.10,0.742857,0.185714,0.061905,0.004762,0.004762


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

,h1,h2,h3,h4,er
lr,,,,,
0.01,0.995238,0.004762,0.000000,0.00000,0.000000
0.02,0.957143,0.038095,0.004762,0.00000,0.000000
0.05,0.928571,0.066667,0.004762,0.00000,0.000000
0.10,0.790476,0.142857,0.033333,0.02381,0.009524
